In [1]:
from modules.binanceklines.binance_klines import *

from modules.postgrescrud.postgres_crud import PostgresCrud as PG
from datetime import datetime

In [2]:
asset_symbol = 'BTCUSDT'
candle_interval = '1m'
start_time = '1241893500000'
max_attempts = 10

binance_klines = GetKlines(asset_symbol, candle_interval, start_time, max_attempts)

In [3]:
binance_klines

In [4]:
binance_klines.formated()

[[datetime.datetime(2017, 8, 17, 4, 0),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  1.775183],
 [datetime.datetime(2017, 8, 17, 4, 1),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 2),
  4280.56,
  4280.56,
  4280.56,
  4280.56,
  0.261074],
 [datetime.datetime(2017, 8, 17, 4, 3),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.012008],
 [datetime.datetime(2017, 8, 17, 4, 4),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.140796],
 [datetime.datetime(2017, 8, 17, 4, 5),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 6),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 7),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 8),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 9),
  4261.48,
  4261.48,
  4261.48,
  4261.48,
  0.0],
 [datetime.datetime(2017, 8, 17, 4, 10),
  4261.48,
  4261.4

In [ ]:
DB_HOST='localhost'
POSTGRES_USER='pandazord'
POSTGRES_DB='pandazord_database'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'

In [ ]:
pg = PG(DB_HOST, POSTGRES_DB, POSTGRES_USER, POSTGRES_PASSWORD)

In [ ]:
records, status = pg.read_records_from_table('binance_assets')

In [ ]:
records

In [ ]:
update_table_job_status = pg.update_record('binance_assets', 'asset_symbol', 'BTCUSDT', 'status', 'building')

In [ ]:
update_table_job_status

# Atribuições

## main.py

Ciclar lendo a tabela de pares

- A) Para os pares cujo "automatically update" seja "yes":  
    
    
> 1 - "status = "building", verificar se o PID está ativo. Caso não esteja ativo, chamar o "continuous_update_of_historical_data.py" em segundo plano (atualizando a tabela de pares, no respectivo par, com o PID do processo chamado). Caso esteja ativo, bypassar;  
    
 > 2 - "status" = "absent" ou "full", chamar o "continuous_update_of_historical_data.py" em segundo plano (atualizando a tabela de pares, no respectivo par, com o PID do processo chamado).    

- Caso o "automatically update" seja seja "no", bypassar.  

## continuous_update_of_historical_data.py

- A) Buscar a tabela de dados do ativo em questão

> 1 - Tabela ausente: 
>> 1.1 - Cria a tabela;  
>> 1.2 - Puxa a primeira leva de dados junto à binance, com um start_time muito antigo;  
>> 1.3 - Procede a verifiação e eventual correção de dados faltantes;  
>> 1.4 - Salva na respectiva tabela do banco;  
>> 1.5 - Segue para 2;  

> 2 - Tabela presente:
>> 2.1 - A partir do timestamp do último candle salvo, faz a solicitação da próxima leva de dados  
>> 2.2 - Procede a verifiação e eventual correção de dados faltantes;  
>> 2.3 - Salva na respectiva tabela do banco;  
>> 2.4 - Marca os status como "full"

In [ ]:
# update_minute_data.py

#TODO: Docstrings

import os
import sys
import time
from datetime import datetime
import requests
from modules.postgrescrud.postgres_crud import PostgresCrud as PG
from modules.binanceklines.binance_klines import *

#from modules import binance_functions as bf, db_functions as dbf, general_functions as gf

#pair = str(sys.argv[1])
asset_symbol = 'BTCUSDT'

#interval = str(sys.argv[2])
candle_interval = '1m'

#start_time = str(sys.argv[3])
start_time = '1241893500000'

table_name = 'binance_klines_' + pair + '_' + candle_interval

keys = {'open_time': 'timestamp', 'open': 'numeric', 'high': 'numeric', 'low': 'numeric', 'close': 'numeric',
    'volume': 'numeric', 'quote_asset_volume': 'numeric', 'number_of_trades': 'integer',
    'taker_buy_base_asset_volume': 'numeric', 'taker_buy_quote_asset_volume': 'numeric'}

max_attempts = 10 #pode vir de um parâmetro depois

update_table_job_status = pg.update_record('binance_assets', 'asset_symbol', pair, 'status', 'building')

if (update_table_job_status == 'fail'): #Tratar exceção update_table_job_status = 'fail'
    pass

round_counter = 1

while True:

    klines = []
        
    binance_klines = GetKlines(asset_symbol, candle_interval, start_time, max_attempts)
    
    klines = binance_klines.formated()
    
    if (len(klines) > 0):
        
        if (len(klines) < 500): #Chegou na kline mais recente
            
            klines = klines[:(len(klines)-1)] #Apaga o último resgistro (candle não fechado)
            
            if (len(klines) == 0): break
        
        most_recent_open_time = klines[len(klines)-1][0] #open_time do último candle

        start_time = str(most_recent_open_time + 30000) #Um passo de 30s (30000 milissegundos)
               
        save_in_table_job_status = dpg.save_data_in_table(table_name, keys, klines)
        
        data_completer_job_status = 'done'
        
        break

        if (len(klines) > 500): break #TODO: Isto é uma anomalia, deve ser tratada
            
    else: #(Falha de comunicação com a binance: klines nulas)
        
        pass
    
    response_ping = requests.get('https://api.binance.com/api/v1/ping')

    total_req_courrent_minute = int(response_ping.headers['X-MBX-USED-WEIGHT'])
    
    if(total_req_courrent_minute >= 1100): time.sleep(60)

    round_counter+=1

update_table_job_status = pg.update_record('binance_assets', 'asset_symbol', 'BTCUSDT', 'status', 'full')

if (update_table_job_status == 'fail'): pass #Tratar exceção update_table_job_status = 'fail'

In [ ]:
#DRAFT: criando lista auxiliar de open_times

open_time_list_in = []

for i in range(len(klines) - 1):
    
    open_time_entry = klines[i][0]
    
    #print (open_time_entry)
    
    open_time_list_in.append(open_time_entry)

In [ ]:
def aux_open_time(data_in):
    
    aux_open_time_list = []
    
    current_open_time = data_in.loc[0, 'open_time']

    last_open_time = data_in.loc[len(data_in)-1, 'open_time']
    
    while (current_open_time < last_open_time):
        
        aux_open_time_list.append(current_open_time)
        
        current_open_time = datetime.fromtimestamp(datetime.timestamp(current_open_time) + 60)
        
    return aux_open_time_list

In [ ]:
#Para anexar aos testes

from copy import deepcopy

def baguncar_klines(klines_in):
    
    klines_out = deepcopy(klines_in)
    
    for i in range(len(klines_in)):
      
        klines_out[i][0] = (klines_in[i][0] + 40000)
    
    return klines_out

In [ ]:
#Trash

